In [93]:
#-- Modules & Packages

import pandas as pd 
import numpy as np 

from glob import glob 
from time import strftime, sleep 
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay 

import yfinance as yf
yf.pdr_override()

#-- Plotting 

import seaborn as sns
import dash 
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc 
import plotly.express as px 
import dash_table 
import plotly.graph_objects as go 
from jupyter_dash import JupyterDash

In [94]:
# pip install jupyter_dash
# pip install dash_bootstrap_components

This Dashboard relies on synthetic transactions data, recurrent USD amount to purchase each asset at the start of each month since 2022; 
this approach is obviously not the most realistic, as we create a data set of imaginary buy-spot regardless of market conditions without
selLing, the aim is to provide a realistic and full-fledged dashboard to monitor the performance of the portfolio. 


I can build on top of this code by selecting from a transaction file, buy & sell dates,
buy & sell prices, fees, cumulative units, cumulative cost, average price of purchase, average selling price, etc...

<h5>Fake Transactions Dataset.<br>

<h6>Amount per crypto:<br>
BTC-USD 50000 $ <br>
ETH-USD 50000 $ <br>
AVAX-USD 10000 $ <br>
OP-USD 10000 $ <br>
ARB-USD 10000 $.

In [253]:
portfolio_tickers = ['BTC-USD', 'ETH-USD', 'OP-USD','ARB-USD','AVAX-USD']

one_year = datetime.today().year - 1
today = datetime.today()
start_x = datetime(2022, 1, 1)
end_x = today

start_assets = datetime(2022,1,1)
end_assets = today

start_ytd = datetime(one_year, 12, 31) + BDay(1)

# def get_txs_data(tickers, startDate, interval):
def data_x(ticker):
    #-- monthly interval '1mo'
    data = (yf.download(ticker, start = start_x, interval= '1mo'))
    data.columns = [x.lower() for x in data.columns]
    data.drop(columns = ['open','high','low','volume','adj close'], axis = 1, inplace = True)
    data['ticker'] =  ticker
    data['type'] = 'Buy'
    data['val_transact'] = 50000
    data['quantity'] = data['val_transact']/ data['close'] 
    data['prev_units'] = data['quantity'].shift(1)
    data['cml_units'] = data['quantity'].cumsum()
    data['prev_cost'] = data['val_transact'].shift(1)
    data['cml_cost'] = data['val_transact'].cumsum()
    #-- applying spot taker-fees in Binance Spot Markets
    data['cost_transact'] = data['val_transact'] * 0.0009500
    data['cml_invested'] = data['val_transact'].cumsum() - data['cost_transact']
    data['cost_unit'] = data['cml_cost'] / data['quantity']
    data['cum_position_val'] = data['cml_units'] * data['close']
    data['gain_loss'] = data['cum_position_val'] - data['cml_invested'] 
    data['yield'] = (data['gain_loss'] / data['cml_invested']) - 1
    #-- running, cumulative mean to accurately assess the avg price through each purchase date/row
    data['avg_price'] = data['close'].expanding().mean()
    #-- Assigning recurrent exact amount for each crypto asset
    if ticker == 'BTC-USD':
      data['val_transact'] = 50000
    elif ticker == 'ETH-USD':
      data['val_transact'] = 50000
    elif ticker == 'OP-USD':
      data['val_transact'] = 10000
    elif ticker == 'ARB-USD':
      data['val_transact'] = 10000
    elif ticker == 'AVAX-USD':
      data['val_transact'] = 10000
    return data

#-- Fetching data for BTC-USD
btcusd = data_x('BTC-USD')
#-- Fetching data for ETH-USD
ethusd = data_x('ETH-USD')
#-- Fetching data for OP-USD
opusd = data_x('OP-USD')
#-- Fetching data for ARB-USD
arbusd = data_x('ARB-USD')
#-- Fetching data for AVAX-USD
avaxusd = data_x('AVAX-USD')
#-- Fetching data for AVAX-USD
injusd = data_x('INJ-USD')


btcusd
# opusd
ethusd

first_concat =  pd.concat([btcusd, ethusd], axis =1 )
second_concat =  pd.concat([opusd, arbusd], axis = 1)
third_concat = pd.concat([injusd, avaxusd], axis = 1)
all_df = [first_concat,second_concat, third_concat]
transactions_df = pd.concat(all_df)
#-- Saving transactions dataframe
transactions_df.to_csv(r"D:\_datasets\Projects_Coding\portfolio_dashboard_base2\inputs\transactions.xlsx")
#-- Previewing transactions dataframe
transactions_df.tail(5) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,close,ticker,type,val_transact,quantity,prev_units,cml_units,prev_cost,cml_cost,cost_transact,...,cml_units,prev_cost,cml_cost,cost_transact,cml_invested,cost_unit,cum_position_val,gain_loss,yield,avg_price
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-01,35.775787,INJ-USD,Buy,50000.0,1397.593280,2717.490799,378253.490507,50000.0,1200000.0,47.5,...,67079.482160,50000.0,1200000,47.5,1199952.5,925.077850,2.585573e+06,1.385620e+06,0.154729,27.388319
2024-01-01,33.890171,INJ-USD,Buy,50000.0,1475.354017,1397.593280,379728.844524,50000.0,1250000.0,47.5,...,68587.236560,50000.0,1250000,47.5,1249952.5,829.047489,2.274483e+06,1.024531e+06,-0.180344,27.619262
2024-02-01,40.921391,INJ-USD,Buy,50000.0,1221.854862,1475.354017,380950.699386,50000.0,1300000.0,47.5,...,69808.673661,50000.0,1300000,47.5,1299952.5,1064.320053,2.857645e+06,1.557693e+06,0.198269,28.131421
2024-03-01,36.806293,INJ-USD,Buy,50000.0,1358.463330,1221.854862,382309.162716,50000.0,1350000.0,47.5,...,70732.709969,50000.0,1350000,47.5,1349952.5,1460.981552,3.827377e+06,2.477425e+06,0.835194,29.093606
2024-04-01,26.451136,INJ-USD,Buy,50000.0,1890.278009,1358.463330,384199.440724,50000.0,1400000.0,47.5,...,72060.747041,50000.0,1400000,47.5,1399952.5,1054.187439,2.713055e+06,1.313102e+06,-0.062038,29.399176


Data Collecting and Saving.

In [258]:
#-- Cleaning columns string names
def clean_headers(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(','').str.replace(')','').str.replace(' ','_').str.replace('_/_','/')

#-- Getting timestamps for file names before saving 
def get_tmstmp():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

last_file = glob('../inputs/transactions.xlsx')
# print(last_file[-(len(last_file)) + (last_file.rfind('/')+1):])

all_ops = pd.read_csv(last_file)
all_ops.date = pd.to_datetime(all_ops.date, format = "%d/%m/%Y")

all_tickers = list(all_ops['ticker'].unique())
blackList = ['PEPE-USD', 'DOGE-USD', 'ADA-USD']
final_tickers = [tick for tick in all_tickers if tick not in blackList]
print("Traded {} different cryptos").format(len(all_tickers))

final_filtered = all_ops[~all_ops.ticker.isin(blackList)]

: 

In [ ]:
portfolio_tickers = ['BTC-USD', 'ETH-USD', 'ENA-USD', 'OP-USD','ARB-USD','AVAX-USD']

one_year = datetime.today().year - 1
today = datetime.today()
start_x = datetime(2023, 1, 1)
end_x = today

start_assets = datetime(2023,1,1)
end_assets = today

start_ytd = datetime(one_year, 12, 31) + BDay(1)

def get_data(tickers, startDate, endDate):
    def data(ticker):
        data = (pdr.get_data_yahoo(ticker, start = startDate, end = endDate))
        data.columns = [x.lower() for x in data.columns]
        data.drop(columns = ['adj close'], axis = 1, inplace = True)
        return data
    datum = map(data, tickers)
    return(pd.concat(datum, keys = tickers, names = ['ticker', 'date']))

prices_df = get_data(portfolio_tickers, start_x, end_x)
prices_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


open          high           low         close  \
ticker   date                                                                 
BTC-USD  2023-01-01  16547.914062  16630.439453  16521.234375  16625.080078   
         2023-01-02  16625.509766  16759.343750  16572.228516  16688.470703   
         2023-01-03  16688.847656  16760.447266  16622.371094  16679.857422   
         2023-01-04  16680.205078  16964.585938  16667.763672  16863.238281   
         2023-01-05  16863.472656  16884.021484  16790.283203  16836.736328   
...                           ...           ...           ...           ...   
AVAX-USD 2024-04-09     49.883575     49.944057     46.394878     46.470196   
         2024-04-10     46.470482     47.967793     45.674950     47.311695   
         2024-04-11     47.311695     47.482182     45.679466     46.057598   
         2024-04-12     46.057159     46.870613     36.225365     39.417744   
         2024-04-13     39.417908     39.418114     29.727076     34.078411   

                          volume  
ticker   date                     
BTC-USD  2023-01-01   9244361700  
         2023-01-02  12097775227  
         2023-01-03  13903079207  
         2023-01-04  18421743322  
         2023-01-05  13692758566  
...                          ...  
AVAX-USD 2024-04-09    437892500  
         2024-04-10    530754231  
         2024-04-11    371204369  
         2024-04-12    966043351  
         2024-04-13   1507323529  

[1419 rows x 5 columns]

NameError: name 'pd' is not defined

In [ ]:
portfolio_all_values = pd.DataFrame({'assetTicker': ['BTC-USD','ETH-USD', 'ENA-USD', 'AVAX-USD'], 
                                     'quantity':[500, 7500, 1000000,12000 ]})
portfolio_all_values


txs_prices = pd.merge(prices_df, portfolio_all_values, how = 'outer', left_index=True, right_index = True).fillna(0)




ValueError: cannot join with no overlapping index names